# FINTECH OptionPricing Lecture02

In [1]:
from pylab import plt, mpl
%matplotlib inline
import math
import numpy as np

In [2]:
plt.style.use( "seaborn" )
mpl.rcParams[ 'font.family' ] = 'serif'

# Variance Reduction

In [3]:
print('{:15s} {:15s}'.format('mean', 'std.'))
print(31 * '-')
for i in range(1, 31, 2):
    np.random.seed(100)
    sn = np.random.standard_normal(i ** 2 * 10000)
    print('{:15.12f} {:15.12f}'.format( sn.mean(), sn.std()))

mean            std.           
-------------------------------
 0.001150944833  1.006296354600
 0.002841204001  0.995987967146
 0.001998082016  0.997701714233
 0.001322322067  0.997771186968
 0.000592711311  0.998388962646
-0.000339730751  0.998399891450
-0.000228109010  0.998657429396
 0.000295768719  0.998877333340
 0.000257107789  0.999284894532
-0.000357870642  0.999456401088
-0.000528443742  0.999617831131
-0.000300171536  0.999445228838
-0.000162924037  0.999516059328
 0.000135778889  0.999611052522
 0.000182006048  0.999619405229


### Antithetic Variates

In [4]:
print('{:15s} {:15s}'.format( 'mean', 'std.' ))
print(31 * '-')
for i in range(1, 31, 2):
    np.random.seed(100)
    sn = np.random.standard_normal(i ** 2 * 5000)
    sn = np.concatenate((sn, -sn))      # 取另一半的相反数
    print('{:15.12f} {:15.12f}'.format(sn.mean(), sn.std()))

mean            std.           
-------------------------------
-0.000000000000  1.017474904235
 0.000000000000  0.998944776883
 0.000000000000  0.998018870658
 0.000000000000  0.997916558845
-0.000000000000  0.998119811838
 0.000000000000  0.998365584578
 0.000000000000  0.998171301115
-0.000000000000  0.998359568664
-0.000000000000  0.998772594780
-0.000000000000  0.998640403762
 0.000000000000  0.998868127277
 0.000000000000  0.999102053335
 0.000000000000  0.999342143561
 0.000000000000  0.999449673443
-0.000000000000  0.999613960655


### Moment Matching

In [5]:
print('{:15s} {:15s}'.format('mean', 'std.'))
print(31 * '-' )
for i in range(1, 31, 2):
    np.random.seed(100)
    sn = np.random.standard_normal(i**2*10000)
    sn = (sn - sn.mean()) / sn.std()
    print('{:15.12f} {:15.12f}'.format(sn.mean(), sn.std()))

mean            std.           
-------------------------------
-0.000000000000  1.000000000000
 0.000000000000  1.000000000000
-0.000000000000  1.000000000000
-0.000000000000  1.000000000000
-0.000000000000  1.000000000000
-0.000000000000  1.000000000000
 0.000000000000  1.000000000000
 0.000000000000  1.000000000000
-0.000000000000  1.000000000000
-0.000000000000  1.000000000000
 0.000000000000  1.000000000000
 0.000000000000  1.000000000000
-0.000000000000  1.000000000000
-0.000000000000  1.000000000000
 0.000000000000  1.000000000000


## Practice: A function to return variance reduced normal

In [12]:
def get_paths(
        M: int, I: int, anti_path: bool=True, mo_matching: bool=True
) -> np.ndarray:
    
    """
    随机数生成器，可以一次性生成全部所需的随机数
    """
    
    if anti_path:
        sn = np.random.standard_normal((M, int(I / 2)))
        sn = np.concatenate((sn, -sn)) 
        if I % 2 == 0:
            pass
        else:
            sn = np.concatenate((sn, np.random.standard_normal((M, 1))), axis=1)
    else:
        sn = np.random.standard_normal((M, I))
    
    if mo_matching:
        sn = (sn - sn.mean()) / sn.std()
    
    return sn

# European Option

In [7]:
S0 = 100
r = 0.05
sigma = 0.25
T = 2.0
I = 10000
M = 100
K = 105

In [ ]:
def European_Option_Pricer_OneStep(S0, K, T, r, sigma, I, optionType = "Call"):
    sn = get_paths(1, I)
    
    ST = S0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt( T ) * sn[ 0 ])
    if optionType == "Call":
        hT = np.maximum(ST - K, 0)
    else:
        hT = np.maximum(K - ST, 0)
        
    C0 = np.exp(-r * T) * hT.mean()
    return C0

# In class practice: multi-step simulation

# In class exercise: Implement Black-Scholes Model 

$C = N(d_1)S_0 - N(d_2)Ke^{-rT}$

$P = N(-d_2)Ke^{-rT} - N(-d_1)S_0$

$d_1 = \frac{ \ln{\frac{S_0}{K}} + (r + \frac{\sigma^2}{2})T}{ \sigma \sqrt{T}}$

$d_2 = d_1 - \sigma \sqrt{T}$

$C - P = S-Ke^{-rT}$

Hint: use scipy.stats.norm to calculate cdf of normal distribution.

# Binomial Option Pricing Model

$u=e^{\sigma\sqrt{dt}}$

$d=\frac{1}{u}$

$p=\frac{e^{rdt} - d}{u-d}$

In [ ]:
def binomial_tree_pricer(S0, K, T, r, sigma, M):
    """
    Implements the binomial option pricing model to price a European call option on a stock
    S - stock price today
    K - strike price of the option
    T - time until expiry of the option
    r - risk-free interest rate
    vol - the volatility of the stock
    M - number of steps in the model
    """
    dt = T/M
    u =  np.exp(sigma * np.sqrt(dt))
    d = 1/u
    p = (np.exp(r * dt) - d)/(u - d)
    C = {}
    for m in range(0, M+1):
        C[(M, m)] = max(S0 * (u ** (2*m - M)) - K, 0)
    for k in range(M-1, -1, -1):
        for m in range(0,k+1):
            C[(k, m)] = np.exp(-r * dt) * (p * C[(k+1, m+1)] + (1-p) * C[(k+1, m)])
    #print(C)
    return C[(0,0)]

# Call Price Statistics

In [ ]:
S0 = 100
r = 0.05
sigma = 0.25
T = 2.0
I = 50000

M = 50

stats_blackscholes = []
stats_montecarlo_onestep = []
stats_montecarlo_dynamic = []
stats_binomialtree = []
k_values = np.arange( 80, 120, 5 )
np.random.seed( 80 )
for K in k_values:
    stats_blackscholes.append( Black_Scholes_Option_Pricer( S0, K, T, r, sigma ) )
    stats_montecarlo_onestep.append( European_Option_Pricer_OneStep( S0, K, T, r, sigma, I ) )
    stats_montecarlo_dynamic.append( European_Option_Pricer( S0, K, T, r, sigma, M, I ) )
    stats_binomialtree.append( binomial_tree_pricer2( S0, K, T, r, sigma, M ) )

# Take Home:

Trinomial Model:

$u = e^{\sigma\sqrt{2\Delta t}}$

$d = e^{-\sigma\sqrt{2\Delta t}} = \frac{1}{u}$

$m = 1$

$p_u = ( \frac{ e^{r\Delta t /2} - e^{ -\sigma\sqrt{\Delta t/2} } }{ e^{ \sigma\sqrt{\Delta t/2} } - e^{ -\sigma\sqrt{\Delta t/2} } })^2$

$p_d = ( \frac{ e^{ \sigma\sqrt{\Delta t/2} } - e^{-r\Delta t /2} }{ e^{ \sigma\sqrt{\Delta t/2} } - e^{ -\sigma\sqrt{\Delta t/2} } })^2$

$p_m = 1 - (p_u + p_d)$